# INF8111 - Fouille de données


## TP1 ÉTÉ 2020 - Système de recommandation

##### Membres de l'équipe:

    - Kacem Khaled
    - Oumayma Messoussi
    - Semah Aissaoui


## Résumé

*Stack Exchange* est un réseau de sites chacun contenant des discussions sur un sujet. Une discussion, ou *thread* en anglais, contient une question et potentiellement plusieurs réponses et commentaires. Dans ce TP, vous implémenterez un système de recommandations qui retourne les discussions en rapport à une question spécifique. Avant de soumettre une question, le site utilisera ce système de recommandations pour proposer les discussions les plus similaires afin de limiter le nombre de discussions dupliquées.


## 2 - Installation

Pour ce TP, vous aurez besoin des librairies `numpy`, `sklearn` et `scipy` (que vous avez sans doute déjà), ainsi que la librairie `nltk`, qui est une libraire utilisée pour faire du traitement du language (Natural Language Processing, NLP)

Installez les libraires en question et exécutez le code ci-dessous :

In [14]:
# Si vous le souhaitez, vous pouvez utiliser anaconda

# pip install --user numpy
# pip install --user sklearn
# pip install --user scipy
# pip install --user nltk


#python
import numpy as np
import scipy as sp
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to C:\Users\Sameh
[nltk_data]     Aissaoui\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## 3 - Jeu de données

Téléchargez l'archive à l'adresse suivante: https://drive.google.com/file/d/1032N1oZkytHlHs20AXE9jPMBQCTyhb6H/view?usp=sharing

L'archive contient:
1. test.json: Ce fichier contient les nouvelles questions avec les discussions pertinentes associées.
2. threads: Ce dossier contient le code HTML des discussions. Chaque fichier HTML est nommé selon le motif **thread_id.html**.

L'image ci-dessous illustre un exemple de discussion :

![thread_img](thread_example.png)

- A : le sujet de la question
- B : le corps de la question
- C : les commentaires de la question
- D : le corps de la réponse
- E : les commentaires de la réponse



In [15]:
import os

# define the folder path that contain the data
# FOLDER_PATH = "Define folder path that contain threads folder and test.json"
FOLDER_PATH ="C:/Users/Sameh Aissaoui/Desktop/Nouveau PC/Maitrise/Cours/Été 2020/INF8111/Labs/dataset"#"dataset/"
THREAD_FOLDER = os.path.join(FOLDER_PATH, 'threads')


# Load the evaluation dataset
import json


test = json.load(open(os.path.join(FOLDER_PATH, "test.json")))
relevant_threads_by_query = dict()


for (query_id, cand_id, label) in test: 
    if label == 'Irrelevant':
        continue
        
    l = relevant_threads_by_query.setdefault(query_id, [])
    l.append(cand_id)

# 4 - Web scraping

"Le *web scraping* (parfois appelé harvesting) est une technique d'extraction du contenu de sites Web, via un script ou un programme, dans le but de le transformer pour permettre son utilisation dans un autre contexte, par exemple le référencement." [Wikipedia](https://fr.wikipedia.org/wiki/Web_scraping)

## 4.1 - Question 1 (0.5 point)

Les caractères spéciaux et non-ASCII peuvent être encodés avec leur représentation HTML. Par exemple, l'apostrophe (') est encodée par **\&AMP;**. L'encodage des pages web n’est pas uniforme, seuls les caractères spéciaux et non-ASCII sont encodés avec leur représentation HTML. Cela sera rectifié en convertissant les représentations HTML en caractère UTF-8. Par exemple, **\&AMP;** sera remplacé par **'**.

Implémentez la fonction fix_encoding qui convertit les représentations HTML en caractères UTF-8.


In [16]:
def fix_encoding(text):
    """
    Encodes the html entities in a text into UTF-8 encoding. For instance, "I&apos;m ..." => "I'm ..."
    
    :param text: string.
    :return: fixed text(sting)
    """
    text=[text]
    text=[w.replace("&apos;","'")for w in text]
    new_text=''
#     for w in text:
#         new_text+=w
    return new_text.join(text) #new_text
        
    

In [17]:
print(fix_encoding(" I&apos;m Semah Aissaoui and I&apos;m 25 years old, I&apos;m working with oumayma and kacem."))
print(fix_encoding(" "))
print(fix_encoding(" I&apos;m &"))
print(fix_encoding(" &apos; "))


 I'm Semah Aissaoui and I'm 25 years old, I'm working with oumayma and kacem.
 
 I'm &
 ' 


## 4.2 - Question 2 (3 points)

Implémentez la fonction extract_data_from_page. Cette fonction extrait le sujet de la question, son corps, ses commentaires, le corps des réponses et leurs commentaires des pages HTML. La fonction retourne un dictionnaire avec la structure suivante : *{"thread_id": int,"question":{"subject": string, "body": string, "comments": [string]}, answers: [{"body": string, "comments": [string]}]}*

**Utilisez la fonction fix_encoding pour convertir le texte. Vouz pouvez utiliser la bibliothèque [Beatiful Soup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) pour cette question. Vous devez retirer toutes HTML du texte de la question, des commentaires et des réponses. **


In [27]:
import numpy as np
from bs4 import BeautifulSoup
def extract_data_from_page(pagepath):
    """
    Scrap question, answer and comments from thread page.
    
    :param pagepath: the path of thread html file.
    :return: 
        {
            "thread_id": thread id,
            "question":{
                "subject": question subject text (Area A in the figure), 
                "body": question body text (Area B in the figure), 
                "comments": list of comment texts (Area C in the figure)
                }, 
            "answers": [
                {
                    "body": answer body text (Area D in the figure),
                    "comments": list of answer texts (Area E in the figure)
                }
                ]
            }
    """
    soup = BeautifulSoup(open(pagepath,encoding='utf8')) #/100724500520.html:4 answers #/100021749708.html : one answer
#     data={}
#     question_details={}
#     # The thread id
#     data["thread_id"] = pagepath.split('/')[-1][:-5]

#     # The question subject
#     subject=soup.find('a', class_="question-hyperlink").get_text()
#     # print("The subject of the question is : ",subject)
#     question_details["subject"]=subject
#     # The question body
#     body_question=soup.find('div',class_='question').find(class_='post-text').get_text()
#     # print("The body of the question is : ",body_question)
#     question_details["body"]=body_question
#     # The question comments
#     comments=[]
#     for comment in soup.find('div',class_='question').find_all('span',class_='comment-copy'):
#         comments=np.append(comments,comment.get_text())
#     # for c in comments:
#     #     print(c)
#     question_details["comments"]=comments
#     data["question"]=question_details

#     # The answer body    
#     answers=[]
#     for ans in soup.find_all('div',class_='answer'):
#         answer_comments=[]
#         answer={}
#         answer["body"]=ans.find(class_='post-text').get_text()
#     #     print(ans.find(class_='post-text').get_text())
#         # The answer comments
#         for com_ans in ans.find_all('span',class_='comment-copy'):
#             answer_comments=np.append(answer_comments,com_ans.get_text())
#         answer["comments"]=answer_comments
#     #         print(com_ans.get_text())
#         answers=np.append(answers,answer)
#     # print(answers)
#     data["answers"]=answers
    data = {}
    answer = {}
    data['thread_id'] = pagepath.split('/')[-1][:-5]
    data['question'] = {}
    soup = BeautifulSoup(open(pagepath,encoding='utf8'))
    question =  soup.find("div", class_="question")
    answers =  soup.find_all("div", class_="answer")
    data['question']['subject'] = soup.find("a", class_="question-hyperlink").get_text()
    data['question']['body'] = question.find("div", class_="postcell post-layout--right").find("div", class_="post-text").get_text().strip()
    data['question']['comments'] = [s.get_text().strip() for s in question.find_all("span", class_="comment-copy")]
    data['answers'] = []
    for ans in answers:
        answer['body'] = ans.find("div", class_="post-text").get_text().strip()
        answer['comments'] = [s.get_text().strip() for s in ans.find_all("span", class_="comment-copy")]
        data['answers'].append(answer)
        answer = {}
    return data
    

In [28]:
# pagepath=THREAD_FOLDER+'/100724500520.html'
# data_web=extract_data_from_page(pagepath)
# print(data_web["question"]["subject"])
# print(data_web["question"]["body"])
# print(data_web["question"]["comments"])
# print(data_web["answers"][0]["body"])
# print(data_web["answers"][0]["comments"])

## 4.3 - Extraire le texte du code HTML

In [29]:
import os
from multiprocessing import Pool, TimeoutError
from time import time
from tqdm import tqdm
import json
# Index each thread by its id
index_path = os.path.join(THREAD_FOLDER, 'threads.json')
if os.path.isfile(index_path):
    # Load threads that webpage content were already extracted.
    thread_index = json.load(open(index_path))
else:
    # Extract webpage content
    # This can be slow (around 30 minutes). Test your code with a small sample. lxml parse is faster than html.parser
    files = (os.path.join(THREAD_FOLDER, filename) for filename in os.listdir(THREAD_FOLDER))
    threads = map(extract_data_from_page, files)
    print('Ok')
    thread_index = dict(((thread['thread_id'], thread) for thread in tqdm(threads)))
    # Save preprocessed threads
    json.dump(thread_index, open(index_path,'w'))
    

Ok



0it [00:00, ?it/s]
1it [00:00,  9.69it/s]
2it [00:00,  6.58it/s]
4it [00:00,  7.42it/s]
6it [00:00,  8.33it/s]
7it [00:00,  8.63it/s]
9it [00:01,  9.41it/s]
10it [00:01,  7.46it/s]
11it [00:01,  7.63it/s]
13it [00:01,  8.50it/s]
15it [00:01,  9.24it/s]
17it [00:01,  8.06it/s]
19it [00:02,  8.73it/s]
20it [00:02,  9.03it/s]
22it [00:02,  9.48it/s]
24it [00:02, 10.10it/s]
26it [00:02,  8.49it/s]
28it [00:03,  8.84it/s]
30it [00:03,  9.29it/s]
31it [00:03,  9.44it/s]
33it [00:03,  8.64it/s]
34it [00:03,  8.94it/s]
35it [00:03,  8.84it/s]
36it [00:04,  7.78it/s]
37it [00:04,  7.74it/s]
39it [00:04,  7.42it/s]
41it [00:04,  8.16it/s]
42it [00:04,  8.45it/s]
43it [00:04,  8.56it/s]
44it [00:05,  8.24it/s]
45it [00:05,  7.84it/s]
47it [00:05,  7.57it/s]
48it [00:05,  8.02it/s]
49it [00:05,  8.08it/s]
51it [00:05,  8.69it/s]
53it [00:06,  9.46it/s]
54it [00:06,  6.91it/s]
55it [00:06,  7.60it/s]
57it [00:06,  8.40it/s]
58it [00:06,  8.72it/s]
59it [00:06,  9.03it/s]
60it [00:06,  8.99it/s]
61

693it [01:45,  6.43it/s]
694it [01:46,  5.04it/s]
695it [01:46,  5.52it/s]
696it [01:46,  5.54it/s]
697it [01:46,  5.88it/s]
698it [01:46,  6.01it/s]
699it [01:47,  6.14it/s]
700it [01:47,  6.65it/s]
701it [01:47,  5.05it/s]
702it [01:47,  5.70it/s]
703it [01:47,  6.50it/s]
704it [01:47,  6.47it/s]
705it [01:48,  6.77it/s]
706it [01:48,  7.09it/s]
707it [01:48,  6.64it/s]
708it [01:48,  6.57it/s]
709it [01:48,  5.40it/s]
710it [01:48,  5.80it/s]
712it [01:49,  6.34it/s]
713it [01:49,  6.80it/s]
714it [01:49,  6.88it/s]
715it [01:49,  6.75it/s]
716it [01:49,  7.04it/s]
717it [01:49,  5.50it/s]
718it [01:50,  5.84it/s]
719it [01:50,  6.24it/s]
720it [01:50,  6.26it/s]
721it [01:50,  6.52it/s]
722it [01:50,  6.43it/s]
723it [01:50,  6.83it/s]
724it [01:51,  6.17it/s]
725it [01:51,  4.98it/s]
726it [01:51,  5.37it/s]
727it [01:51,  5.86it/s]
728it [01:51,  5.99it/s]
729it [01:51,  6.54it/s]
730it [01:51,  7.23it/s]
731it [01:52,  6.91it/s]
732it [01:52,  7.02it/s]
733it [01:52,  5.34it/s]


1339it [03:32,  6.83it/s]
1340it [03:32,  7.04it/s]
1341it [03:32,  5.36it/s]
1342it [03:33,  5.74it/s]
1343it [03:33,  5.83it/s]
1344it [03:33,  6.53it/s]
1345it [03:33,  6.60it/s]
1346it [03:33,  6.37it/s]
1347it [03:33,  6.32it/s]
1348it [03:33,  6.24it/s]
1349it [03:34,  4.75it/s]
1350it [03:34,  5.12it/s]
1351it [03:34,  5.59it/s]
1352it [03:34,  5.95it/s]
1353it [03:34,  6.30it/s]
1354it [03:34,  6.48it/s]
1355it [03:35,  6.46it/s]
1356it [03:35,  4.94it/s]
1357it [03:35,  5.20it/s]
1358it [03:35,  5.56it/s]
1359it [03:35,  6.22it/s]
1360it [03:36,  6.12it/s]
1361it [03:36,  6.09it/s]
1362it [03:36,  6.37it/s]
1363it [03:36,  6.60it/s]
1364it [03:36,  6.83it/s]
1365it [03:36,  5.47it/s]
1366it [03:37,  5.88it/s]
1367it [03:37,  6.32it/s]
1368it [03:37,  6.08it/s]
1369it [03:37,  5.44it/s]
1370it [03:37,  5.71it/s]
1371it [03:37,  5.95it/s]
1372it [03:38,  6.39it/s]
1373it [03:38,  4.90it/s]
1374it [03:38,  5.30it/s]
1375it [03:38,  5.92it/s]
1376it [03:38,  6.34it/s]
1377it [03:3

1976it [05:18,  5.89it/s]
1977it [05:18,  6.36it/s]
1978it [05:18,  6.42it/s]
1979it [05:18,  6.21it/s]
1980it [05:18,  6.36it/s]
1981it [05:18,  6.49it/s]
1982it [05:18,  6.90it/s]
1983it [05:19,  5.20it/s]
1984it [05:19,  5.68it/s]
1985it [05:19,  5.46it/s]
1986it [05:19,  5.90it/s]
1987it [05:19,  6.35it/s]
1988it [05:19,  6.77it/s]
1989it [05:20,  7.03it/s]
1990it [05:20,  6.77it/s]
1991it [05:20,  4.81it/s]
1992it [05:20,  5.27it/s]
1993it [05:20,  5.90it/s]
1994it [05:21,  5.71it/s]
1995it [05:21,  5.92it/s]
1996it [05:21,  6.39it/s]
1997it [05:21,  6.82it/s]
1998it [05:21,  6.93it/s]
1999it [05:21,  6.87it/s]
2000it [05:21,  7.00it/s]
2001it [05:22,  5.31it/s]
2002it [05:22,  5.54it/s]
2003it [05:22,  6.13it/s]
2004it [05:22,  5.89it/s]
2005it [05:22,  6.29it/s]
2006it [05:22,  6.43it/s]
2007it [05:23,  6.82it/s]
2008it [05:23,  7.35it/s]
2009it [05:23,  5.36it/s]
2010it [05:23,  5.64it/s]
2011it [05:23,  5.97it/s]
2012it [05:23,  6.43it/s]
2013it [05:24,  6.76it/s]
2014it [05:2

...

25561it [1:12:07,  7.53it/s]
25562it [1:12:08,  4.43it/s]
25563it [1:12:08,  4.95it/s]
25564it [1:12:08,  5.19it/s]
25566it [1:12:08,  5.94it/s]
25567it [1:12:09,  6.18it/s]
25568it [1:12:09,  6.01it/s]
25569it [1:12:09,  6.10it/s]
25570it [1:12:09,  6.27it/s]
25571it [1:12:09,  5.92it/s]
25572it [1:12:09,  6.68it/s]
25573it [1:12:10,  4.23it/s]
25575it [1:12:10,  5.03it/s]
25576it [1:12:10,  5.50it/s]
25577it [1:12:10,  5.50it/s]
25578it [1:12:10,  6.14it/s]
25579it [1:12:11,  6.19it/s]
25580it [1:12:11,  6.43it/s]
25581it [1:12:11,  7.02it/s]
25583it [1:12:11,  5.59it/s]
25584it [1:12:12,  6.21it/s]
25585it [1:12:12,  6.15it/s]
25586it [1:12:12,  6.41it/s]
25587it [1:12:12,  6.32it/s]
25588it [1:12:12,  6.41it/s]
25589it [1:12:12,  6.06it/s]
25590it [1:12:12,  6.22it/s]
25591it [1:12:13,  6.35it/s]
25592it [1:12:13,  6.58it/s]
25593it [1:12:13,  6.65it/s]
25594it [1:12:13,  4.09it/s]
25595it [1:12:14,  4.74it/s]
25596it [1:12:14,  5.30it/s]
25597it [1:12:14,  5.73it/s]
25598it [1:12:

26140it [1:13:49,  5.90it/s]
26141it [1:13:49,  6.15it/s]
26142it [1:13:49,  6.21it/s]
26143it [1:13:49,  6.80it/s]
26144it [1:13:49,  7.19it/s]
26145it [1:13:49,  6.86it/s]
26146it [1:13:50,  6.80it/s]
26148it [1:13:50,  6.89it/s]
26149it [1:13:50,  6.83it/s]
26150it [1:13:50,  4.02it/s]
26151it [1:13:51,  4.15it/s]
26153it [1:13:51,  4.88it/s]
26154it [1:13:51,  5.49it/s]
26155it [1:13:51,  5.99it/s]
26156it [1:13:51,  6.80it/s]
26157it [1:13:51,  6.35it/s]
26158it [1:13:52,  6.71it/s]
26159it [1:13:52,  6.95it/s]
26160it [1:13:52,  7.55it/s]
26161it [1:13:52,  3.85it/s]
26162it [1:13:53,  4.58it/s]
26163it [1:13:53,  5.04it/s]
26164it [1:13:53,  5.22it/s]
26165it [1:13:53,  5.75it/s]
26166it [1:13:53,  5.81it/s]
26167it [1:13:53,  5.70it/s]
26169it [1:13:54,  6.29it/s]
26170it [1:13:54,  6.03it/s]
26171it [1:13:54,  6.43it/s]
26172it [1:13:54,  4.08it/s]
26173it [1:13:54,  4.93it/s]
26174it [1:13:55,  5.49it/s]
26175it [1:13:55,  5.02it/s]
26176it [1:13:55,  5.47it/s]
26177it [1:13:

26721it [1:15:31,  6.26it/s]
26722it [1:15:31,  6.28it/s]
26723it [1:15:31,  6.72it/s]
26724it [1:15:31,  6.57it/s]
26725it [1:15:31,  6.99it/s]
26726it [1:15:32,  7.32it/s]
26727it [1:15:32,  7.23it/s]
26728it [1:15:32,  7.44it/s]
26729it [1:15:32,  4.67it/s]
26731it [1:15:32,  5.60it/s]
26732it [1:15:33,  6.41it/s]
26733it [1:15:33,  7.02it/s]
26734it [1:15:33,  7.23it/s]
26735it [1:15:33,  7.77it/s]
26737it [1:15:33,  8.59it/s]
26739it [1:15:33,  8.63it/s]
26740it [1:15:33,  8.08it/s]
26741it [1:15:34,  4.45it/s]
26742it [1:15:34,  4.45it/s]
26744it [1:15:34,  5.21it/s]
26745it [1:15:34,  5.84it/s]
26746it [1:15:35,  6.27it/s]
26747it [1:15:35,  6.46it/s]
26748it [1:15:35,  6.62it/s]
26749it [1:15:35,  6.20it/s]
26750it [1:15:35,  5.99it/s]
26751it [1:15:35,  6.04it/s]
26752it [1:15:36,  3.84it/s]
26753it [1:15:36,  4.32it/s]
26754it [1:15:36,  4.94it/s]
26755it [1:15:36,  5.60it/s]
26756it [1:15:36,  5.92it/s]
26757it [1:15:37,  6.20it/s]
26758it [1:15:37,  6.12it/s]
26759it [1:15:

27302it [1:17:11,  4.75it/s]
27303it [1:17:11,  5.29it/s]
27304it [1:17:11,  5.91it/s]
27305it [1:17:11,  6.00it/s]
27306it [1:17:12,  6.09it/s]
27307it [1:17:12,  6.58it/s]
27308it [1:17:12,  6.35it/s]
27309it [1:17:12,  6.44it/s]
27310it [1:17:12,  6.59it/s]
27311it [1:17:12,  6.80it/s]
27312it [1:17:12,  6.61it/s]
27313it [1:17:13,  3.94it/s]
27314it [1:17:13,  4.23it/s]
27315it [1:17:13,  4.89it/s]
27316it [1:17:13,  5.61it/s]
27317it [1:17:14,  6.10it/s]
27318it [1:17:14,  6.06it/s]
27319it [1:17:14,  6.42it/s]
27321it [1:17:14,  6.94it/s]
27322it [1:17:14,  6.80it/s]
27323it [1:17:14,  6.83it/s]
27324it [1:17:15,  6.92it/s]
27325it [1:17:15,  4.31it/s]
27326it [1:17:15,  4.94it/s]
27327it [1:17:15,  5.39it/s]
27328it [1:17:15,  5.74it/s]
27329it [1:17:15,  6.33it/s]
27330it [1:17:16,  6.22it/s]
27331it [1:17:16,  6.13it/s]
27332it [1:17:16,  6.75it/s]
27333it [1:17:16,  6.86it/s]
27334it [1:17:16,  6.85it/s]
27335it [1:17:16,  7.11it/s]
27336it [1:17:16,  7.44it/s]
27337it [1:17:

27877it [1:18:51,  6.73it/s]
27878it [1:18:52,  4.05it/s]
27880it [1:18:52,  4.72it/s]
27881it [1:18:52,  5.18it/s]
27882it [1:18:52,  5.54it/s]
27883it [1:18:53,  6.00it/s]
27884it [1:18:53,  5.89it/s]
27885it [1:18:53,  6.32it/s]
27886it [1:18:53,  6.92it/s]
27887it [1:18:53,  6.71it/s]
27888it [1:18:54,  3.95it/s]
27889it [1:18:54,  4.42it/s]
27890it [1:18:54,  4.91it/s]
27891it [1:18:54,  5.75it/s]
27892it [1:18:54,  6.50it/s]
27893it [1:18:54,  6.70it/s]
27894it [1:18:55,  6.79it/s]
27895it [1:18:55,  6.11it/s]
27896it [1:18:55,  6.45it/s]
27897it [1:18:55,  6.38it/s]
27898it [1:18:55,  6.43it/s]
27899it [1:18:55,  6.46it/s]
27900it [1:18:55,  6.44it/s]
27901it [1:18:56,  3.95it/s]
27902it [1:18:56,  4.30it/s]
27903it [1:18:56,  4.68it/s]
27904it [1:18:56,  4.96it/s]
27905it [1:18:57,  5.14it/s]
27906it [1:18:57,  4.68it/s]
27907it [1:18:57,  4.97it/s]
27908it [1:18:57,  5.57it/s]
27909it [1:18:57,  5.97it/s]
27910it [1:18:58,  3.83it/s]
27911it [1:18:58,  4.65it/s]
27912it [1:18:

## 5 - Prétraitement des données

Le prétraitement des données est une tache cruciale en fouille de données. Cette étape nettoie et transforme les données brutes dans un format qui permet leur analyse, et leur utilisation avec des algorithmes de *machine learning*. En traitement des langages (natural language processing, NLP), la *tokenization* et le *stemming* sont des étapes cruciales. De plus, vous implémenterez une étape supplémentaire pour filtrer les mots sans importance.

### 5.1 - Tokenization

Cette étape permet de séparer un texte en séquence de *tokens* (= jetons, ici des mots, symboles ou ponctuation).

Par exemple, la phrase *"It's the student's notebook."* peut être séparé en liste de tokens de cette manière: ["it", " 's", "the", "student", " 's", "notebook", "."].

**De plus, tous les tokenizers ont également le rôle de mettre le texte en minuscule.**

#### 5.1.1 - Question 3 (0.5 point) 

Implémentez la fonction suivante :

- **tokenize_space** qui tokenize le texte à partir des blancs (espace, tabulation, nouvelle ligne). Ce tokenizer est naïf.
- **tokenize_nltk** qui utilise le tokenizer par défaut de la librairie nltk (https://www.nltk.org/api/nltk.html).



In [33]:
from nltk.tokenize import WhitespaceTokenizer
from nltk.tokenize import word_tokenize
def tokenize_space(text):

    """
    Tokenize the tokens that are separated by whitespace (space, tab, newline). 
    We consider that any tokenization was applied in the text when we use this tokenizer.
    
    For example: "hello\tworld of\nNLP" is split in ['hello', 'world', 'of', 'NLP']
    """
    tokenized_text=text.split()
    # return a list of tokens
    return tokenized_text
        
def tokenize_nltk(text):
    """
    This tokenizer uses the default function of nltk package (https://www.nltk.org/api/nltk.html) to tokenize the text.
    """
    # return a list of tokens
    return word_tokenize(text)#WhitespaceTokenizer().tokenize(text)
    
    
        

In [34]:
text="hello\tworld of\nNLP"
print(tokenize_space(text))
print(tokenize_nltk(text))

['hello', 'world', 'of', 'NLP']
['hello', 'world', 'of', 'NLP']


### 5.2 - Filtrer les tokens sans importance

#### 5.2.1 - Question 4 (1 point)

Certains tokens sont sans importance pour la comparaison, car ils apparaissent dans la majorité des discussions. Les supprimer réduit la dimension du vecteur et accélère les calculs.

Expliquez quels tokens sont sans importances pour la comparaison des discussions. Implémentez la fonction filter_tokens qui retire ces mots de la liste des tokens.


In [38]:
from nltk.corpus import stopwords
nltk.download('stopwords')
def filter_tokens(tokens):
    tokens_without_sw = [word for word in tokens if not word in stopwords.words()]
    return tokens_without_sw


[nltk_data] Downloading package stopwords to C:\Users\Sameh
[nltk_data]     Aissaoui\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [40]:
print(stopwords.words())

['إذ', 'إذا', 'إذما', 'إذن', 'أف', 'أقل', 'أكثر', 'ألا', 'إلا', 'التي', 'الذي', 'الذين', 'اللاتي', 'اللائي', 'اللتان', 'اللتيا', 'اللتين', 'اللذان', 'اللذين', 'اللواتي', 'إلى', 'إليك', 'إليكم', 'إليكما', 'إليكن', 'أم', 'أما', 'أما', 'إما', 'أن', 'إن', 'إنا', 'أنا', 'أنت', 'أنتم', 'أنتما', 'أنتن', 'إنما', 'إنه', 'أنى', 'أنى', 'آه', 'آها', 'أو', 'أولاء', 'أولئك', 'أوه', 'آي', 'أي', 'أيها', 'إي', 'أين', 'أين', 'أينما', 'إيه', 'بخ', 'بس', 'بعد', 'بعض', 'بك', 'بكم', 'بكم', 'بكما', 'بكن', 'بل', 'بلى', 'بما', 'بماذا', 'بمن', 'بنا', 'به', 'بها', 'بهم', 'بهما', 'بهن', 'بي', 'بين', 'بيد', 'تلك', 'تلكم', 'تلكما', 'ته', 'تي', 'تين', 'تينك', 'ثم', 'ثمة', 'حاشا', 'حبذا', 'حتى', 'حيث', 'حيثما', 'حين', 'خلا', 'دون', 'ذا', 'ذات', 'ذاك', 'ذان', 'ذانك', 'ذلك', 'ذلكم', 'ذلكما', 'ذلكن', 'ذه', 'ذو', 'ذوا', 'ذواتا', 'ذواتي', 'ذي', 'ذين', 'ذينك', 'ريث', 'سوف', 'سوى', 'شتان', 'عدا', 'عسى', 'عل', 'على', 'عليك', 'عليه', 'عما', 'عن', 'عند', 'غير', 'فإذا', 'فإن', 'فلا', 'فمن', 'في', 'فيم', 'فيما', 'فيه', 'فيها', '

In [41]:
tx="Nick likes to play football, however he is not too fond of tennis."
tok_tx=tokenize_nltk(tx)
print(tok_tx)
print(filter_tokens(tok_tx))


['Nick', 'likes', 'to', 'play', 'football', ',', 'however', 'he', 'is', 'not', 'too', 'fond', 'of', 'tennis', '.']
['Nick', 'likes', 'play', 'football', ',', 'however', 'fond', 'tennis', '.']


### 5.3 - Stemming

La racinisation (stemming) est un procédé de transformation des flexions en leur radical ou racine. Par example, en anglais, la racinisation de "fishing", "fished" and "fish" donne "fish" (stem). 



In [42]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")



word1 = ["Visitors", "from", "all", "over", "the", "world", "fishes", "during", "the", "summer","."]

print([ stemmer.stem(w) for w in word1])

word2 = ['I', 'was', 'fishing',]
print([ stemmer.stem(w) for w in word2])

['visitor', 'from', 'all', 'over', 'the', 'world', 'fish', 'dure', 'the', 'summer', '.']
['i', 'was', 'fish']


#### 5.3.1 - Question 5 (1 point) 

*Expliquez comment et pourquoi le stemming est utile à note système de comparaison.*


Le stemming est utile pour notre comparaison puisque en regardant deux sujets, on peut trouver deux mots qui ont été écrites de façons différentes mais ils ont le même racine tels que : poisson et pêche. Alors pour distinguer si deux mots ont le même sens ou la même racine, on applique le stemming pour comparer directement les mots par leurs racines.

# 6 - Data representation

## 6.1 - Bag of Words

De nombreux algorithmes demande des entrées qui soient toutes de la même taille, ce qui n'est forcément le cas pour des types de données comme les textes, qui peuvent avoir un nombre variable de mots.  

Par exemple, considérons la phrase 1, ”Board games are much better than video games” et la phrase 2, ”Monopoly is an awesome game!”. La table ci-dessous montre un exemple d'un moyen de représentation de ces deux phrases en utilisant une représentation fixe : 

|<i></i>     | an | are | ! | Monopoly | awesome | better | games | than | video | much | board | is | game |
|------------|----|-----|---|----------|---------|--------|-------|------|-------|------|-------|----|------|
| Sentence 1 | 0  | 1   | 0 | 0        | 0       | 1      | 2     | 1    | 1     | 1    | 1     | 0  | 0    |
| Sentence 2 | 1  | 0   | 1 | 1        | 1       | 0      | 0     | 0    | 0     | 0    | 0     | 1  | 1    |

Chaque colonne représente un mot du vocabulaire (de longueur 13), tandis que chaque ligne contient l'occurence des mots dans une phrase. Ainsi, la valeur 2 à la position (1,7) est due au fait que le mot *"games"* apparait deux fois dans la phrase 1. 

Ainsi, chaque ligne étant de longueur 13, on peut les utiliser comme vecteur pour représenter les phrases 1 et 2. Ainsi, c'est cette méthode que l'on appelle *Bag-of-Words* : c'est une représentation de documents par des vecteurs dont la dimension est égale à la taille du vocabulaire, et qui est construit en comptant le nombre d'occurence de chaque mot. Ainsi, chaque token est ici associé à une dimension.

### 6.1.2 - Question 6 (2.5 points)


*Implémentez le Bag-of-Words*

Pour cette question, vous ne pouvez pas utiliser de librairie Python externe comme scikit-learn, hormis si vous avez des problèmes de mémoire, vous pouvez utiliser la classe sparse.csr_matrix de scipy (https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csc_matrix.html).

In [1]:
import numpy as np
from scipy.sparse import csc_matrix

In [54]:
def transform_count_bow(X):
    """
    This method preprocesses the data using the pipeline object, relates each token to a specific integer and  
    transforms the text in a vector. Vectors are weighted using the token frequencies in the sentence.

    X: document tokens. e.g: [['I','will', 'be', 'back', '.'], ['Helllo', 'world', '!'], ['If', 'you', 'insist', 'on', 'using', 'a', 'damp', 'cloth']]

    :return: vector representation of each document
    """   
    words=[]
    for sentence  in X:
        for word in sentence:
            if word not in words:
                words.append(word)
    sentence_number=len(X)
    data=[]
    for sentence in X:
        occurence=[0]*len(words)
        for w in words:
            if w in sentence:
                index = words.index(w)
                freq=sentence.count(w)
                occurence[index]=freq
        data.append(occurence)
    data=np.asarray(data)
    
    return data

In [53]:
import re
a=[['I','will', 'be', 'back', '!'], ['Helllo', 'world', '!'], ['If', 'you', 'insist', 'on', 'using', 'a', 'damp', 'cloth']]

print(len(transform_count_bow(a)[1]))
print(len(transform_count_bow(a)[0][0]))
print(transform_count_bow(a)[1])
print(transform_count_bow(a)[0])



15
15
['I', 'will', 'be', 'back', '!', 'Helllo', 'world', 'If', 'you', 'insist', 'on', 'using', 'a', 'damp', 'cloth']
[[1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 1 1 1 1 1 1]]


## 6.2 - TF-IDF

L'utilisation de la fréquence d'apparition brute des mots, comme c'est le cas avec le bag-of-words, peut être problématique. En effet, peu de tokens auront une fréquence très élevée dans un document, et de ce fait, le poids de ces mots sera beaucoup plus grand que les autres, ce qui aura tendance à biaiser l'ensemble des poids. De plus, les mots qui apparaissent dans la plupart des documents n'aident pas à les discriminer. Par exemple, le mot "*de*" apparaît dans beaucoup de documents de la base de données, et pour autant, avoir ce mot en commun ne permet pas de conclure que des documents sont similaires. Au contraire, le mot "*génial*" est plus rare, mais les documents qui contiennent ce mot sont plus susceptibles d'être positif. TF-IDF est donc une méthode qui permet de pallier à ce problème.

TF-IDF pondère le vecteur en utilisant une fréquence de document inverse (IDF) et une fréquence de termes (TF).

TF est l'information locale sur l'importance qu'a un mot dans un document donné, tandis que IDF mesure la capacité de discrimination des mots dans un jeu de données. 

L'IDF d'un mot se calcule de la façon suivante:

\begin{equation}
	\text{idf}_i = \log\left( \frac{N}{\text{df}_i} \right),
\end{equation}

avec $N$ le nombre de documents dans la base de donnée, et $\text{df}_i$ le nombre de documents qui contiennent le mot $i$.

Le nouveau poids $w_{ij}$ d'un mot $i$ dans un document $j$ peut ensuite être calculé de la façon suivante:

\begin{equation}
	w_{ij} = \text{tf}_{ij} \times \text{idf}_i,
\end{equation}

avec $\text{tf}_{ij}$ la fréquence du mot $i$ dans le document $j$.



### 6.2.1 - Question 7 (3.5 points)

Implémentez le bag-of-words avec la pondération de TF-IDF

Pour cette question, vous ne pouvez pas utiliser de librairie Python externe comme scikit-learn, hormis si vous avez des problèmes de mémoire, vous pouvez utiliser la classe sparse.csr_matrix de scipy (https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csc_matrix.html).

In [86]:
import math 
def transform_tf_idf_bow(X):
    """
    This method preprocesses the data using the pipeline object, calculates the IDF and TF and 
    transforms the text in vectors. Vectors are weighted using TF-IDF method.

    X: document tokens. e.g: [['I','will', 'be', 'back', '.'], ['Helllo', 'world', '!'], ['If', 'you', 'insist', 'on', 'using', 'a', 'damp', 'cloth']]

    :return: vector representation of each document
    """
    data=transform_count_bow(X)
    N=data.shape[0]
    W=np.zeros(shape = (N, data.shape[1]))
    for i in range(data.shape[1]):
        df=np.count_nonzero(data[:,i])
        IDF=math.log(N/df,10)
        W[:,i]=data[:,i]*IDF
    return W

In [89]:
print(transform_tf_idf_bow(a))

[[0.47712125 0.47712125 0.47712125 0.47712125 0.17609126 0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.17609126 0.47712125
  0.47712125 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.47712125 0.47712125 0.47712125 0.47712125 0.47712125
  0.47712125 0.47712125 0.47712125]]


# 7 - Système de recommandations

## 7.1 - Question 8 (1.5 points)


La *pipeline* est la séquence d'étapes de prétraitement des données qui transforme les données brutes dans un format qui permet leur analyse. Pour le problème de système de recommandations, implémentez un pipeline composé des étapes suivantes :

1. Concatène le texte de la question, des réponses et des commentaires pour chaque discussion $t$ dans le dictionnaire thread_dict.
2. Tokenize le texte.
3. Filtre les tokens sans importance.
4. Stem les tokens
5. Génère la représentation vectorielle avec transform_tf_idf_bow ou transform_count_bow.
6. Retourne les identifiants des discussions et les représentations vectorielles des discussions. 



In [ ]:
def nlp_pipeline(thread_dict, tokenization_type, vectorizer_type, enable_filter_tokens, enable_stemming):
    """
    Preprocess and vectorize the threads.
    
    thread_dict: dictionary whose keys and values are thread ids and thread objects, respectively.
    tokenization_type: two possible values "space_tokenization" and "nltk_tokenization".
                            - space_tokenization: tokenize_space function is used to tokenize.
                            - nltk_tokenization: tokenize_nltk function is used to tokenize.
                            
    vectorizer_type: two possible values "count" and "tf_idf".
                            - count: use transform_count_bow to vectorize the text
                            - tf_idf: use transform_tf_idf_bow to vectorize the text
                            
    enable_filter_tokens: enable the insignificant token removal;
    
    enable_stemming: enable stemming
    
    return: a list L with thread ids and matrix B that contains the vector of each thread. B[idx] is the fixed-length representation of L[idx].
    """
    
    t
    
    raise NotImplementedError("")
    

## 7.2 - Question 9 (1.5 points)

Implémentez la fonction rank qui retourne la liste des identifiants des discussions triés par leur similarité avec la nouvelle question (query). Vous utiliserez la [cosine similarity function](https://en.wikipedia.org/wiki/Cosine_similarity) pour comparer deux discussions. Considérez la nouvelle question comme une discussion sans réponse ni commentaire.

**Retirez la nouvelle question dans la liste de recommandations**




In [ ]:
def rank(query_id, all_thread_ids, X):
    """
    Return a list of thread ids sorted by thread and query similarity. Cosine similarity is used to compare threads. 
    
    query_id: thread id 
    all_thread_ids: list of thread ids
    X: thread data representations
    
    return: ranked list of thread ids. 
    """
    
    # Compute the similarity of thread representations(vectors) using cosine similarity function
    # Sort the thread ids by the similarity
    
    raise NotImplementedError("")
    

## 7.3 - Évaluation

Vous allez tester différentes configurations du système de recommandations. Ces configurations seront comparées avec la [mean average precision (MAP) metric](https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)#Mean_average_precision). Plus les discussions pertinentes sont recommandées rapidement (c.-à-d. en haut de la liste), plus élevé sera le score MAP. 

Ressources supplémentaires pour comprendre MAP: [recall and precision over ranks](https://youtu.be/H7oAofuZjjE) et [MAP](https://youtu.be/pM6DJ0ZZee0).


La fonction *eval* évalue une configuration spécifique du système de recommandations.

In [24]:
from statistics import mean 


def calculate_map(x):
    res = 0.0
    n = 0.0
    
    
    for relevant_threads, ranked_list in x:
        precisions = []
               
        for k, thread_id in enumerate(ranked_list):
            if thread_id in relevant_threads:
                prec_at_k = (len(precisions) + 1)/(k+1)
                precisions.append(prec_at_k)
                
            if len(precisions) == len(relevant_threads):
                break
        res += mean(precisions)
        n += 1
    
    return res/n
            

def eval(tokenization_type, vectorizer, enable_filter_tokens, enable_stemming):
    all_thread_ids, X = nlp_pipeline(thread_index, tokenization_type, vectorizer, enable_filter_tokens, enable_stemming)
    all_thread_ids = [int(t_id) for t_id in all_thread_ids]    
    queries,relevant_threads = zip(*relevant_threads_by_query.items())
        
    def generate_rank_list(query_id):
        return rank(query_id, all_thread_ids, X)
    
    with Pool(processes=2) as pool:
        ranked_list = pool.map(generate_rank_list, queries)
        
        
    return calculate_map(zip(relevant_threads,ranked_list))
        

## 7.4 - Question 10 (5 points)

Évaluez la précision (MAP) du système de recommandations avec chacune des configurations suivantes :
1. count(BoW) + space_tokenization (sans tokenizer)
2. count(BoW) + nltk_tokenization
3. count(BoW) + nltk_tokenization + Filtrer les tokens sans importance
4. count(BoW) + nltk_tokenization + Filtrer les tokens sans importance + Stemming
5. tf_idf + nltk_tokenization
6. tf_idf + nltk_tokenization + Filtrer les tokens sans importance
7. tf_idf + nltk_tokenization + Filtrer les tokens sans importance + Stemming 

Enfin, commentez vos résultats en répondant aux questions suivantes :
- Notre système de recommandation a-t-il été influencé négativement ou positivement par les étapes de prétraitement des données ??
- Est-ce que TF-IDF est plus performant que BoW? Si oui, à votre avis, pourquoi?
